In [24]:
#pragma cling add_include_path("/usr/local/include/eigen3")
#pragma cling add_include_path("/home/amir/Robotics/Mujoco/Projects/ModernRoboticsCpp/include/")
#pragma cling load("/usr/local/lib/libModernRoboticsCpp.so") // Ensure the shared library is loaded


#include <iostream>
#include <Eigen/Dense>
#include "modern_robotics.h"



In [25]:
int main() {
 // Define the end-effector configuration at home position (M)
  Eigen::Matrix4d M;
  M << 1, 0, 0, 2,
       0, 1, 0, 0,
       0, 0, 1, 1,
       0, 0, 0, 1;

  // Define the list of screw axes for each joint in space frame (Slist)
  Eigen::MatrixXd Slist(6, 3);
  Slist << 0, 0, 1,
           0, 1, 0,
           1, 0, 0,
           0, 0, 0,
           0, 0, 0,
           0, 0, 0;

  // Define the joint angles (thetaList)
  Eigen::VectorXd thetaList(3);
  thetaList << M_PI / 2, M_PI / 4, M_PI / 6;

    // Compute the forward kinematics
    Eigen::Matrix4d T = mr::FKinSpace(M, Slist, thetaList);

    // Output the result
    std::cout << "The end-effector configuration is:\n" << T << std::endl;

    return 0;
}

In [26]:
main();

The end-effector configuration is:
7.85046e-17   -0.866025         0.5         0.5
   0.707107    0.353553    0.612372     2.02659
  -0.707107    0.353553    0.612372   -0.801841
          0           0           0           1


In [27]:
// Function to check if a value is effectively zero
bool isEffectivelyZero(double value, double threshold = 1e-9) {
    return std::abs(value) < threshold;
}

In [28]:
#include <iostream>
#include <Eigen/Dense>
#include "modern_robotics.h"




// Function to calculate the screw axis for a joint given its type, axis direction, and a point on the axis
Eigen::VectorXd CalculateScrewAxis(const std::string& joint_type, const Eigen::Vector3d& axis_direction, 
                                   const Eigen::Vector3d& point_on_axis) {
  Eigen::VectorXd screw_axis(6); // Initialize screw axis as 6-vector

  if (joint_type == "revolute") {
    screw_axis.head(3) = axis_direction.normalized(); // ωi: Normalized rotation axis
    screw_axis.tail(3) = -axis_direction.cross(point_on_axis); // Use the temporary 3D vector for the cross product
  } else if (joint_type == "prismatic") {
    screw_axis.head(3) = Eigen::Vector3d::Zero(); // ωi: Zero for prismatic joints
    screw_axis.tail(3) = axis_direction.normalized(); // vi: Normalized translation direction
  } else {
    std::cerr << "Error: Invalid joint type. Must be 'revolute' or 'prismatic'." << std::endl;
    return Eigen::VectorXd::Zero(6); // Return a zero vector on error
  }

  for (int i = 0; i < 6; ++i) {
    if (isEffectivelyZero(screw_axis(i))) {
      screw_axis(i) = 0.0; // Set effectively zero values to zero
    }
  }

  return screw_axis.transpose(); // Return the screw axis as a row vector
}


In [29]:
// Function to calculate and store screw axes in a matrix
Eigen::MatrixXd ScrewMat(const std::vector<Eigen::Vector3d>& joint_axes, 
                         const std::vector<Eigen::Vector3d>& joint_points, 
                         const std::vector<std::string>& joint_types) {
  // Number of joints and size of screw axis
  int joint_num = joint_axes.size();
  const int screw_axes_size = 6;

  // Initialize the matrix to store screw axes
  Eigen::MatrixXd Slist(joint_num, screw_axes_size);

  // Loop to calculate and store screw axes
  for (int i = 0; i < joint_num; ++i) {
    Eigen::VectorXd S = CalculateScrewAxis(joint_types[i], joint_axes[i], joint_points[i]);
    Slist.row(i) = S;
    std::cout << "S" << i + 1 << ": " << S.transpose() << std::endl;
  }

  // Print the entire Slist matrix
  std::cout << "Slist matrix:\n" << Slist << std::endl;

  return Slist;
}

In [30]:
bool NearZero(const double val)
    {
        return (std::abs(val) < .000001);
    }



In [31]:
// Function to calculate the screw axis for a joint given its type, axis direction, and a point on the axis
    Eigen::VectorXd CalculateScrewAxis(const std::string &joint_type, const Eigen::Vector3d &axis_direction,
                                       const Eigen::Vector3d &point_on_axis)
    {
        Eigen::VectorXd screw_axis(6); // Initialize screw axis as 6-vector

        if (joint_type == "revolute")
        {
            screw_axis.head(3) = axis_direction.normalized();          // ωi: Normalized rotation axis
            screw_axis.tail(3) = -axis_direction.cross(point_on_axis); // Use the temporary 3D vector for the cross product
        }
        else if (joint_type == "prismatic")
        {
            screw_axis.head(3) = Eigen::Vector3d::Zero();     // ωi: Zero for prismatic joints
            screw_axis.tail(3) = axis_direction.normalized(); // vi: Normalized translation direction
        }
        else
        {
            std::cerr << "Error: Invalid joint type. Must be 'revolute' or 'prismatic'." << std::endl;
            return Eigen::VectorXd::Zero(6); // Return a zero vector on error
        }

        // Example logic for adjusting values
        for (auto &value : screw_axis)
        {
            if (NearZero(value))
            {
                value = 0.0; // Adjusting the value to be exactly zero
            }
        }
        
        //print the screw axis
        std::cout << "Screw axis: " << screw_axis.transpose() << std::endl;

        return screw_axis;
    }

In [32]:

    // Function to calculate and store screw axes in a matrix
    Eigen::MatrixXd ScrewMat(const std::vector<Eigen::Vector3d> &joint_axes,
                             const std::vector<Eigen::Vector3d> &joint_points,
                             const std::vector<std::string> &joint_types)
    {
        // Number of joints and size of screw axis
        int joint_num = joint_axes.size();
        const int screw_axes_size = 6;

        // Initialize the matrix to store screw axes
        Eigen::MatrixXd Slist(screw_axes_size,joint_num);

        // Loop to calculate and store screw axes
        for (int i = 0; i < joint_num; ++i)
        {
            Eigen::VectorXd S = CalculateScrewAxis(joint_types[i], joint_axes[i], joint_points[i]);
            Slist.col(i) = S;
            std::cout << "S" << i + 1 << ": " << S.transpose() << std::endl;
        }

        // Print the entire Slist matrix
        std::cout << "Slist matrix:\n"
                  << Slist << std::endl;

        return Slist;
    }

In [33]:
void Test1() {
  //UR5 6R robot arm

  
    // Define joint axes
    std::vector<Eigen::Vector3d> joint_axes = {
        {0, 0, 1},
        {0, 1, 0},
        {0, 1, 0},
        {0, 1, 0},
        {0, 0, -1},
        {0, 1, 0}
    };
    double W1=0.109, W2=0.082, L1=0.425, L2=0.392 ,H1=0.089, H2=0.095;

    Eigen::Matrix4d M;
    M << 1, 0, 0, L1+L2,
        0, 1, 0, W1+W2,
        0, 1, 0, H1-H2,
        0, 0, 0, 1;
     // Define points on the joint axes
    std::vector<Eigen::Vector3d> joint_points = {
        {0, 0, 0},                // joint1
        {0, 0, H1},          // joint2
        {L1, 0,H1}, // joint3
        {L1+L2, 0, H1}, // joint4
        {L1+L2, W1, 0}, // joint5
        {L1+L2, 0,H1-H2} // joint6
    };


    // Define joint types
    std::vector<std::string> joint_types = {
        "revolute",
        "revolute",
        "revolute",
        "revolute",
        "revolute",
        "revolute"
    };

    // Call the ScrewMat function
    Eigen::MatrixXd Slist = ScrewMat(joint_axes, joint_points, joint_types);

    
    // Joint angles
    Eigen::VectorXd thetaList(6);
    thetaList << 0, -M_PI/2, 0, 0, M_PI/2, 0;


    Eigen::Matrix4d T=mr::FKinSpace(M,Slist, thetaList);

    std::cout << "The end-effector configuration is:\n" << T << std::endl;
}

In [34]:
Test1();

Screw axis: 0 0 1 0 0 0
S1: 0 0 1 0 0 0
Screw axis:      0      1      0 -0.089      0      0
S2:      0      1      0 -0.089      0      0
Screw axis:      0      1      0 -0.089      0  0.425
S3:      0      1      0 -0.089      0  0.425
Screw axis:      0      1      0 -0.089      0  0.817
S4:      0      1      0 -0.089      0  0.817
Screw axis:      0      0     -1 -0.109  0.817      0
S5:      0      0     -1 -0.109  0.817      0
Screw axis:     0     1     0 0.006     0 0.817
S6:     0     1     0 0.006     0 0.817
Slist matrix:
     0      0      0      0      0      0
     0      1      1      1      0      1
     1      0      0      0     -1      0
     0 -0.089 -0.089 -0.089 -0.109  0.006
     0      0      0      0  0.817      0
     0      0  0.425  0.817      0  0.817
The end-effector configuration is:
 1.2326e-32          -1           0       0.095
         -1 1.11022e-16           0       0.109
1.11022e-16           1           0       0.988
          0           0    